In [1]:
import pickle
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorchtools
import glob as gl
import random
import os
import time
import itertools

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.datasets import make_regression
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import log_loss
from numpy import hstack
from numpy import vstack

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()
find_directory(back_num = 1, foldername = ['Dataset'], filename = 'bat_dict.pkl')

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\All dataset'

'C:\\Users\\User\\Desktop\\LIB\\CNN work\\Variable time length\\Dataset\\bat_dict.pkl'

In [3]:
bat_dict_add_1_2 = find_directory(back_num = 2, foldername = ['Dataset'], filename = 'bat_1_2_dict_VITQcQd.pkl')
with open(bat_dict_add_1_2, 'rb') as tf:
    bat_dict_1_2 = pickle.load(tf)

bat_sel_dict = bat_dict_1_2
max_time = 120

## Split per batch

In [4]:
# batch1: 0~40(41)
# batch2: 41~83(43)
# batch3: 64~123(40)

b1 = 0
b2 = 0
b3 = 0

for cell in bat_sel_dict.keys():
    if cell.startswith('b1'):
        b1 += 1
    elif cell.startswith('b2'):
        b2 += 1
    else:
        b3 += 1

print(b1, b2, b3)

# Tensor dimension: (cell, cycle, time, variable)
regression_cycles = [100]

# variable = [Qc, I, V, T, Qd]
num_variables = len(bat_sel_dict['b1c0']['1']['1.0'].keys())

for reg in regression_cycles:
    globals()[f'x_tensor_{reg}'] = torch.zeros(b1+b2+b3, reg, max_time, num_variables)
    cell_index = 1
    for cell in bat_sel_dict.keys():
        for cycle in range(1, reg+1):
            for time in bat_sel_dict[cell][str(cycle)].keys():
    #             print(bat_sel_dict[cell][str(cycle)][str(time)])
                value_list = [i for i in bat_sel_dict[cell][str(cycle)][str(time)].values()]
    #             print(value_list)
                globals()[f'x_tensor_{reg}'][cell_index-1, cycle-1, int(float(time)*max_time/60), :] = torch.FloatTensor(value_list)

        cell_index += 1
    
    globals()[f'x_tensor_{reg}'].size()

# globals()[f'x_tensor_b1_{reg}'] = globals()[f'x_tensor_{reg}'][:b1, :, :, :]
# globals()[f'x_tensor_b2_{reg}'] = globals()[f'x_tensor_{reg}'][b1:b1+b2, :, :, :]
# globals()[f'x_tensor_b3_{reg}'] = globals()[f'x_tensor_{reg}'][b1+b2:, :, :, :]
    
# globals()[f'x_tensor_b1_{reg}'].size()
# globals()[f'x_tensor_b2_{reg}'].size()
# globals()[f'x_tensor_b3_{reg}'].size()

num_variables = x_tensor_100.shape[3]
time_length = x_tensor_100.shape[2]
# (cycle_index-1, cycle-1, time-1, variable(Qc, I, V, T, Qd))
# x_tensor[123, 99, 30, :]

41 43 40


torch.Size([124, 100, 120, 5])

In [5]:
y_df_add = find_directory(back_num = 2, foldername = ['Dataset', 'y_df.csv'])
y_df = pd.read_csv(y_df_add)

# 2. Regression problem
# KO, KP, CL
y_cl_numpy = y_df['Cycle life'].to_numpy()
y_kp_numpy = y_df['kneepoints'].to_numpy()
y_ko_numpy = y_df['kneeonsets'].to_numpy()
# y_ko_numpy.shape

y_regression_numpy = y_ko_numpy

y_ko_tensor = torch.FloatTensor(y_ko_numpy)
# y_ko_tensor.size()

# y_ko_b1_tensor = y_ko_tensor[:b1]
# y_ko_b2_tensor = y_ko_tensor[b1:b1+b2]
# y_ko_b3_tensor = y_ko_tensor[b1+b2:]
# y_ko_b1_tensor.size()
# y_ko_b2_tensor.size()
# y_ko_b3_tensor.size()

In [6]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed) # torch 
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True # cudnn
    torch.backends.cudnn.benchmark = False # cudnn
    np.random.seed(random_seed) # numpy
    random.seed(random_seed) # random

In [7]:
# y: no scaling
class dataPrep_RNN_CNN(Dataset):  
    def __init__(self, x_tensor, y_tensor, batch_size, scaler, b1, b2, b3):
        setRandomSeed(random_seed = 100)
        self.xdata = torch.permute(x_tensor, (0, 3, 2, 1))
        self.ydata = y_tensor
        
        self.batch_size = batch_size
                
        self.xscaler = scaler()
        
        # Tensor dimension: (cell, variable, time, cycle)
        # If problem emerges, check this part first.
        for i in range(len(self.xdata)):
            for j in range(self.xdata.shape[1]):
                temp = np.expand_dims(self.xscaler.fit_transform(self.xdata[i,j,:,:]), axis=0)
                if j ==0:
                    temp2 = temp
                else:
                    temp2 = np.vstack((temp2,temp))
            temp2 = np.expand_dims(temp2, axis=0)
            if i==0:
                self.xdata_scaled = temp2
            else:
                self.xdata_scaled = np.vstack((self.xdata_scaled, temp2))

        self.ydata_scaled = self.ydata
        
        self.all = list(zip(self.xdata_scaled, self.ydata_scaled))
        
        self.b1 = self.all[:b1]
        self.b2 = self.all[b1:b1+b2]
        self.b3 = self.all[b1+b2:]
        
        self.b1_dataloader = DataLoader(self.b1, batch_size = self.batch_size, shuffle = True)
        self.b2_dataloader = DataLoader(self.b2, batch_size = self.batch_size, shuffle = True)
        self.b3_dataloader = DataLoader(self.b3, batch_size = self.batch_size, shuffle = True)
        
        print(f"batch 1: {len(self.b1)}, batch 2: {len(self.b2)}, batch 3: {len(self.b3)}")
    
    def batch_dataloader(self):
        return self.b1_dataloader, self.b2_dataloader, self.b3_dataloader
    
    def scaler(self):
        return self.xscaler
    
    # finding length of x 
    def __len__(self):
        return len(self.X)
    
    # indexing rows for calling
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

In [8]:
regression_cycles = [100]


# x와 y가 1대1 대응이어야 dataloader를 만들 수 있어서 2D array 그대로 놓고 학습시 tensor transform 예정
for reg in regression_cycles:
    globals()[f'reg_dataset_{reg}'] = dataPrep_RNN_CNN(globals()[f'x_tensor_{reg}'], y_ko_tensor, 4, MinMaxScaler, b1, b2, b3)
    globals()[f'b1_dataloader_{reg}'], globals()[f'b2_dataloader_{reg}'], globals()[f'b3_dataloader_{reg}'] = globals()[f'reg_dataset_{reg}'].batch_dataloader()
    globals()[f'reg_xscaler_{reg}'] = globals()[f'reg_dataset_{reg}'].scaler()

batch 1: 41, batch 2: 43, batch 3: 40


## Define model class

In [9]:
class RNNcell(nn.Module):
    def __init__(self, model, input_size, hidden_size, dropout = 0.1, bidirectional = False, num_layer=1):
        super(RNNcell, self).__init__()
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
            
        self.model = model
        self.hidden_size = hidden_size
        self.num_layer = num_layer
        self.bidirectional = bidirectional
        
        self.D = 1 + self.bidirectional
        self.batch_size = 4
        
        #Batch size * cycles, timestep, input_size
        if self.model.endswith('RNN'):
            self.rnn = nn.RNN(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('LSTM'):
            self.rnn = nn.LSTM(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
        elif self.model.endswith('GRU'):
            self.rnn = nn.GRU(input_size, hidden_size, num_layer, batch_first = True, dropout = dropout, bidirectional = bidirectional).to(self.device)
            
        for name, param in self.rnn.named_parameters():
            if name.startswith('weight'):
                nn.init.xavier_uniform(param)
            else:
                nn.init.normal(param)
        
    def forward(self, x):
        # x made from (cell * n_cy, timestep(seq_len), num_vars)
        self.batch_size = x.size(0)
        
        D = 1 + self.bidirectional
        #out: Containing output features h_t from the last layer of LSTM for each t(if bidirectional ==True: concat of forward and backward hidden states)
        #h_n: Containing final hidden state in the sequence
        h0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
        if self.model.endswith('LSTM'):
            c0 = torch.zeros(D*self.num_layer, self.batch_size, self.hidden_size).to(self.device)
            # c_n: Containing final cell state in the sequence
            out, (hn, cn) = self.rnn(x, (h0, c0))
        else:
            out, hn = self.rnn(x, h0.detach())
        return out, hn

In [10]:
class SelfAttention(nn.Module):
    def __init__(self, hidden_size, num_heads):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size // num_heads
        
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, inputs):
        # inputs shape: [batch_size, seq_len, hidden_size]
        batch_size, seq_len, hidden_size = inputs.size()
        
        # Compute query, key, and value matrices
        query = self.query(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        key = self.key(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        value = self.value(inputs).view(batch_size, seq_len, self.num_heads, self.head_size).transpose(1, 2)
        
        # Compute scaled dot-product attention scores
        # attention_scores: [batch_size, num_heads, seq_len, seq_len]
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.head_size, dtype=torch.float32))
        attention_probs = self.softmax(attention_scores)
        
        # Apply attention to the value matrix and concatenate heads
        # context: [batch_size, num_heads, seq_len, head_size]=>[batch_size, seq_len, num_heads, head_size] 
        # => [batch_size, seq_len, num_heads * head_size]
        context = torch.matmul(attention_probs, value).transpose(1, 2).contiguous().view(batch_size, seq_len, -1)
#         context = nn.Linear(hidden_size, 1)(context)
        return attention_probs, context

In [11]:
class RNN_CA_1DCNN(nn.Module):
    def __init__(self, input_size, num_time, num_cycles, rnn1, bi1, hid1, nhead2, fil2, pool2, npool2, fsize2, psize2, mids, dr1 = 0.1, dr2 = 0.1, di2 = 1, st2 = 1, pad2 = 0):
        super(RNN_CA_1DCNN, self).__init__()
        setRandomSeed()
        
        self.D1 = 1 + bi1
        
        self.rnn1 = RNNcell(rnn1, input_size, hid1, dr1, bi1)
        
        if torch.cuda.is_available():
            self.device = 'cuda'
        else:
            self.device = 'cpu'
        
        # Overall parameters
        self.num_time = num_time
        self.num_cycles = num_cycles
        self.mids = mids
        
        # Pooling/Nonpooling parameters
        # Pooling/Nonpooling layers
        self.pool2 = pool2
        self.npool2 = npool2
        # Pooling size(Pooling layer)
        self.psize2 = psize2
        
        # Params for self-attention(ca)
        self.nh2 = nhead2
        
        # CNN hyperparameters
        self.fil2 = fil2
        self.fsize2 = fsize2
        self.di2 = di2
        self.st2 = st2
        # Padding for CNN
        self.pad2 = pad2
                
        # Critical values for the 1st CNN layer
        # Batch for different cells
        
        # Conv1d takes input dimension: [Batch_size, c_in(no. of in_channels )= hidden_size, l_in = num_cycles]
        self.in2 = self.D1*hid1
        self.out2 = self.fil2
        self.Lout = self.num_cycles
        
        # Conv_block1~P
        # Pooling layers
        for i in range(1, self.pool2+1):
#             print(f"i: {i}, out_channels: {self.out_channels}")
            globals()[f'self.conv_block{i}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            nn.MaxPool1d(self.psize2)
            ).to(self.device)

            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int(int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)/self.psize2)

        for j in range(self.pool2+1, self.pool2+self.npool2+1):
            globals()[f'self.conv_block{j}'] = nn.Sequential(
            nn.Conv1d(self.in2, self.out2, self.fsize2, self.st2, self.pad2),
            nn.BatchNorm1d(self.out2),
            nn.ReLU(),
            ).to(self.device)
            
            self.in2 = self.out2
            self.out2 = self.out2*2
            self.Lout = int((self.Lout+2*self.pad2-self.di2*(self.fsize2-1)-1)/self.st2+1)
        
        self.lin1 = nn.Linear(self.D1*hid1, 1)
#         self.sm1 = nn.Softmax(dim = 2)
        self.relu = nn.ReLU()
        
        self.ca = SelfAttention(self.D1*hid1, self.nh2)
        
        self.fc = nn.Sequential(nn.Linear(self.in2*self.Lout, self.mids[0], bias=True), nn.ReLU(),
                       nn.Linear(self.mids[0], self.mids[1], bias = True), nn.ReLU(),
                       nn.Linear(self.mids[1], self.mids[2], bias = True))
        
        self.fc.apply(self.init_weights)
        
    def init_weights(self, m):
        self.m = m
        if type(self.m) == nn.Linear:
            nn.init.xavier_uniform_(self.m.weight)
        
    def forward(self, x):
        # Original: Batch size, num_vars, timestep, cycles=>Batch size * cycles, timestep, input_size
        x = torch.reshape(x, (x.size(0)*x.size(3), x.size(2), x.size(1))).to(device)
#         print("Batch size * cycles, timestep, input_size: ", x.size())
        out1, hn1 = self.rnn1(x)
        
        # hn의 batch size가 n_cy*k로 설정
        cell_batch_size = int(hn1.size(1)/self.num_cycles)

        # Reshape for input
        # out1: (batch_size = num_cell*num_cycle, timestep, D1*hid1)
        # outs : (batch_size(num_cell), num_cycle, timestep, D1*hid1)
        outs1 = torch.reshape(out1, (cell_batch_size, -1, out1.size(1), out1.size(2)))
        # outs : (batch_size(num_cell), D1*hid1, num_cycle, timestep)
        outs1 = torch.permute(outs1, (0, 3, 1, 2))
        # ta: (batch_size, num_cycle, timestep, 1(D1*hid1->1))
#         ta = self.sm1(self.lin1(outs1))
        # ta_outs: (batch_size, num_cycle, timestep, D1*hid1)=>(batch_size, D1*hid1, num_cycle, timestep)
#         ta_outs = torch.permute(ta*outs1, (0, 3, 1, 2))
        # ct_vec: (batch_size, D1*hid1, num_cycle)
#         ct_vec = torch.sum(ta_outs, -1)
        # ct_vec: (batch_size, D1*hid1, num_cycle)
#         print(outs1.size())
        ct_vec = outs1[:, :, :, -1].squeeze(dim = -1)
#         print(ct_vec.size())
        # ct_vec shape: [batch_size, num_cycle, D1*hid1(hidden_size)]
        # hidden_size = num_heads * head_size
        # ct_vec transform back: [batch_size, num_heads, num_cycle, head_size]=>[batch_size, num_cycle, num_heads, head_size] 
        # => [batch_size, num_cycle, num_heads * head_size(hidden_size)]
        # ca: matmul(query[batch_size, num_heads, num_cycle, head_size], key^T[batch_size, num_heads, head_size, num_cycle])
        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        ca, ct_vec = self.ca(ct_vec.transpose(2, 1))
#         print(f"After self attention: ca.size: {ca.size()}, ct_vec.size: {ct_vec.size()}")
        # ct_vec shape: [batch_size, D1*hid1, num_cycle]
        ct_vec = ct_vec.transpose(2, 1)
        
        
        # input for 1d cnn: (batch_size, D1*hid1, num_cycle)
        for i in range(1, self.pool2+self.npool2+1):
            ct_vec = globals()[f'self.conv_block{i}'](ct_vec)
        # ct_vec from final convolutional layer: (batch_size, self.in2*self.Lout)
        ct_vec = ct_vec.view(ct_vec.size(0), -1)
        # final_out: (batch_size, 1)
        final_out = self.fc(ct_vec)
        
        return final_out.squeeze(), ca

In [12]:
# Function for saving and loading of training history
def save_data(D3_array, filename):
    with open(filename,"wb") as dat_:
        pickle.dump(D3_array,dat_)
        
def load_data(filename):
    with open(filename,"rb") as ld:
        x_temp = pickle.load(ld)
    return x_temp

def history_state_dict_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    history_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'train_history'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}.pkl')
    state_dict_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'model'], 
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_state_dict.pth')
    ca_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca'],
                                             filename = f'{rnn1}_hidden_{hid1}_nh_{nh2}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_ta.wb')
    return history_add, state_dict_add, ca_add

def ca_per_batches_add(num_cycles, modelname, n_ep, patience, rnn1, hid1, nh2, fil2, pool2, npool2, fsize2, psize2, lr):
    batch1_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca_batch1', f'{nh2} heads'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_n_head_{nh2}.pkl')
    batch2_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca_batch2', f'{nh2} heads'], 
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_n_head_{nh2}.pkl')
    batch3_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{rnn1}_1D CNN', 'ca_batch3', f'{nh2} heads'],
                                             filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_n_head_{nh2}.pkl')

    fig_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, 'figures', 'ca', f'{nh2} heads'],
                                                 filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_n_head_{nh2}_1th head.png')
    fig_add = [fig_add]
    if nh2>1:
        for nh in range(1, nh2):
            globals()[f'fig_add_{nh}'] = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, 'figures', 'ca', f'{nh2} heads'],
                                                     filename = f'{rnn1}_hidden_{hid1}_n_fil_{fil2}_pool_{pool2}_npool_{npool2}_fsize_{fsize2}_psize_{psize2}_lr_1_{int(1/lr)}_n_head_{nh2}_{nh+1}th head.png')
            fig_add.append(globals()[f'fig_add_{nh}'])
    
    return batch1_add, batch2_add, batch3_add, fig_add

In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [14]:
print(ca.size())

NameError: name 'ca' is not defined

In [ ]:
num_vars = 5
modelname = 'RNN_CA_1DCNN'
rnns = ['BiLSTM', 'BiGRU', 'BiRNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]
num_heads = [1, 5]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]
batch_size = 4

case_num = 0
case_test_under_100 = 0

b1dl = globals()[f'b1_dataloader_{num_cycles}']
b2dl = globals()[f'b2_dataloader_{num_cycles}']
b3dl = globals()[f'b3_dataloader_{num_cycles}']
dataloaders = [b1dl, b2dl, b3dl]
batches = ['batch1', 'batch2', 'batch3']

print(modelname)
for [n_ep, patience] in ep_pats:
    case_ep = 0
    print(f"epoch = {n_ep}, patience = {patience}")
    for num_head in num_heads:
        case_ep_head = 0
        case_test_under_100_n_ep_num_head = 0
        for rnn1, hid1 in itertools.product(rnns, hids):
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            for pool2, npool2 in itertools.product(pools, npools):
                for fil2, lr in itertools.product(num_fils, lrs):
                    case_num += 1
                    case_ep_head += 1;
                    
                    history_add, state_dict_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)

                    ca_batch1_add, ca_batch2_add, ca_batch3_add, ca_fig_add = ca_per_batches_add(num_cycles, modelname, n_ep, patience,
                                                                    rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)
                    
                    all_hid1 = hid1 * num_head
                    
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, num_head, fil2, pool2, npool2, fsize2, psize2, mids).to(device);
                    _ = model.load_state_dict(load_data(state_dict_add));

                    history = load_data(history_add);
                    rmse = [round(history.iloc[-1, 0],2), round(history.iloc[-1, 1], 2), round(history.iloc[-1, 2], 2)];
                    
                    if round(history.iloc[-1, 2], 2)<100:
#                         print(f'Epoch: {n_ep}, Pateince: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Nonpool: {npool2}, Num_head : {num_head}, Filter: {fil2}, lr: {lr}, RMSE: {rmse}')
                        case_test_under_100 += 1
                        case_test_under_100_n_ep_num_head += 1
                        # Temporal code for 1000 epochs
                        with torch.no_grad():
                            i = 0
                            for nh in range(num_head):
                                globals()[f'fig{nh+2}'], globals()[f'ax{nh+2}'] = plt.subplots(1, 3, figsize = (30, 8));
                            rows = 1
                            cols = 3

                            # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
                            b1_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            b2_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            b3_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            cas = [b1_ca, b2_ca, b3_ca]

                            for dataloader in dataloaders:
                                for n, (inputs, targets) in enumerate(dataloader):
                                    inputs = inputs.float().to(device)
                                    targets = targets.float().to(device)
                                    # ta: (batch_size, num_cycle, timestep)
                                    # ca: (batch_size, num_heads, num_cycle, num_cycle)
                                    yhat, ca = model(inputs)

                                    if len(ca.size()) !=4:
                                        ca = torch.unsqueeze(ca, 0)
                                    cas[i] = torch.cat((cas[i], ca), dim = 0)

                                cas[i] = torch.mean(cas[i][batch_size:], dim = 0).squeeze(dim = -1)
#                                 cas[i].size()
                                cas[i] = cas[i].detach().cpu().numpy();
#                                 print(cas[i].type())
                                
#                                 globals()[f'im_{i}'] = axs[i].imshow(tas[i], vmin = 0, vmax = 0.06, cmap = cm.gray);
                                for nh in range(num_head):
#                                     print(cas[i][nh,:,:])
                                    globals()[f'im{nh+2}_{i}'] = globals()[f'ax{nh+2}'][i].imshow(cas[i][nh,:,:], vmin = 0, vmax = 0.06, cmap = cm.gray);
                                    _ = globals()[f'ax{nh+2}'][i].set_xlabel('Cycle', fontsize = 20);
                                    for t in globals()[f'ax{nh+2}'][i].get_xticklabels():
                                        t.set_fontsize(20);
                                    _ = globals()[f'ax{nh+2}'][i].set_ylabel('Cycle', fontsize = 20);
                                    for t in globals()[f'ax{nh+2}'][i].get_yticklabels():
                                        t.set_fontsize(20);

                                    _ = globals()[f'ax{nh+2}'][i].set_title(f'Batch {i+1} cyclic attention score', fontsize = 25);

                                i+=1

                            for nh in range(num_head):
                                globals()[f'fig{nh+2}'].subplots_adjust(right = 0.85);
                                globals()[f'cbar_ax{nh+2}'] = globals()[f'fig{nh+2}'].add_axes([0.88, 0.15, 0.02, 0.7]);
                            i -= 1
                            
#                             globals()[f'cbar_{i}'] = plt.colorbar(globals()[f'im_{i}'], cax = cbar_ax);
                            for nh in range(num_head):
                                globals()[f'cbar{nh+2}_{i}'] = plt.colorbar(globals()[f'im{nh+2}_{i}'], cax = globals()[f'cbar_ax{nh+2}']);
                                for t in globals()[f'cbar{nh+2}_{i}'].ax.get_yticklabels():
                                    t.set_fontsize(20);

#                             for t in globals()[f'cbar_{i}'].ax.get_yticklabels():
#                                 t.set_fontsize(20);


                        #     plt.colorbar();

                            for nh in range(num_head):
                                _ = globals()[f'fig{nh+2}'].suptitle(f'Epoch: {n_ep}, Patience: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Npool: {npool2}, Filter: {fil2}, lr: {lr}, RMSE:{rmse}, n_head = {num_head}, {nh+1}th head', fontsize = 25);
#                                 globals()[f'fig{nh+2}'].show();
                                plt.close(globals()[f'fig{nh+2}']);
                                globals()[f'fig{nh+2}'].savefig(ca_fig_add[nh]);

#             print(f"{n_ep},{num_head} done")
        print(f"Test error under 100 in n_ep = {n_ep}, n_head = {num_head}: {case_test_under_100_n_ep_num_head}/{case_ep_head}")  
print(f"Test set error under 100 in all cases: {case_test_under_100}/{case_num}")

In [17]:
num_vars = 5
modelname = 'RNN_CA_1DCNN'
rnns = ['LSTM', 'GRU', 'RNN']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]
num_heads = [1, 2, 3, 4, 5]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]
batch_size = 4

case_num = 0
case_test_under_100 = 0

b1dl = globals()[f'b1_dataloader_{num_cycles}']
b2dl = globals()[f'b2_dataloader_{num_cycles}']
b3dl = globals()[f'b3_dataloader_{num_cycles}']
dataloaders = [b1dl, b2dl, b3dl]
batches = ['batch1', 'batch2', 'batch3']

print(modelname)
for [n_ep, patience] in ep_pats:
    case_ep = 0
    print(f"epoch = {n_ep}, patience = {patience}")
    for num_head in num_heads:
        case_ep_head = 0
        case_test_under_100_n_ep_num_head = 0
        for rnn1, hid1 in itertools.product(rnns, hids):
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            for pool2, npool2 in itertools.product(pools, npools):
                for fil2, lr in itertools.product(num_fils, lrs):
                    case_num += 1
                    case_ep_head += 1;
                    
                    history_add, state_dict_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                                rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)

                    ca_batch1_add, ca_batch2_add, ca_batch3_add, ca_fig_add = ca_per_batches_add(num_cycles, modelname, n_ep, patience,
                                                                    rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)
                    
                    all_hid1 = hid1 * num_head
                    
                    model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, num_head, fil2, pool2, npool2, fsize2, psize2, mids).to(device);
                    _ = model.load_state_dict(load_data(state_dict_add));

                    history = load_data(history_add);
                    rmse = [round(history.iloc[-1, 0],2), round(history.iloc[-1, 1], 2), round(history.iloc[-1, 2], 2)];
                    
                    if round(history.iloc[-1, 2], 2)<100:
#                         print(f'Epoch: {n_ep}, Pateince: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Nonpool: {npool2}, Num_head : {num_head}, Filter: {fil2}, lr: {lr}, RMSE: {rmse}')
                        case_test_under_100 += 1
                        case_test_under_100_n_ep_num_head += 1
                        # Temporal code for 1000 epochs
                        with torch.no_grad():
                            i = 0
                            for nh in range(num_head):
                                globals()[f'fig{nh+2}'], globals()[f'ax{nh+2}'] = plt.subplots(1, 3, figsize = (30, 8));
                            rows = 1
                            cols = 3

                            # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
                            b1_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            b2_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            b3_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
                            cas = [b1_ca, b2_ca, b3_ca]

                            for dataloader in dataloaders:
                                for n, (inputs, targets) in enumerate(dataloader):
                                    inputs = inputs.float().to(device)
                                    targets = targets.float().to(device)
                                    # ta: (batch_size, num_cycle, timestep)
                                    # ca: (batch_size, num_heads, num_cycle, num_cycle)
                                    yhat, ca = model(inputs)

                                    if len(ca.size()) !=4:
                                        ca = torch.unsqueeze(ca, 0)
                                    cas[i] = torch.cat((cas[i], ca), dim = 0)

                                cas[i] = torch.mean(cas[i][batch_size:], dim = 0).squeeze(dim = -1)
                                cas[i] = cas[i].detach().cpu().numpy();
                                
                                for nh in range(num_head):
                                    globals()[f'im{nh+2}_{i}'] = globals()[f'ax{nh+2}'][i].imshow(cas[i][nh,:,:], vmin = 0, vmax = 0.06, cmap = cm.gray);
#                                     if num_head>1:
#                                         globals()[f'im{nh+2}_{i}'] = globals()[f'ax{nh+2}'][i].imshow(cas[i][nh,:,:], vmin = 0, vmax = 0.06, cmap = cm.gray);
#                                     else:
#                                         globals()[f'im{nh+2}_{i}'] = globals()[f'ax{nh+2}'][i].imshow(cas[i][nh], vmin = 0, vmax = 0.06, cmap = cm.gray);
                                    _ = globals()[f'ax{nh+2}'][i].set_xlabel('Cycle', fontsize = 20);
                                    for t in globals()[f'ax{nh+2}'][i].get_xticklabels():
                                        t.set_fontsize(20);
                                    _ = globals()[f'ax{nh+2}'][i].set_ylabel('Cycle', fontsize = 20);
                                    for t in globals()[f'ax{nh+2}'][i].get_yticklabels():
                                        t.set_fontsize(20);

                                    _ = globals()[f'ax{nh+2}'][i].set_title(f'Batch {i+1} cyclic attention score', fontsize = 25);

                                i+=1

                            for nh in range(num_head):
                                globals()[f'fig{nh+2}'].subplots_adjust(right = 0.85);
                                globals()[f'cbar_ax{nh+2}'] = globals()[f'fig{nh+2}'].add_axes([0.88, 0.15, 0.02, 0.7]);
                            i -= 1

                            for nh in range(num_head):
                                globals()[f'cbar{nh+2}_{i}'] = plt.colorbar(globals()[f'im{nh+2}_{i}'], cax = globals()[f'cbar_ax{nh+2}']);
                                for t in globals()[f'cbar{nh+2}_{i}'].ax.get_yticklabels():
                                    t.set_fontsize(20);


                            for nh in range(num_head):
                                _ = globals()[f'fig{nh+2}'].suptitle(f'Epoch: {n_ep}, Patience: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Npool: {npool2}, Filter: {fil2}, lr: {lr}, RMSE:{rmse}, n_head = {num_head}, {nh+1}th head', fontsize = 25);
#                                 globals()[f'fig{nh+2}'].show();
                                plt.close(globals()[f'fig{nh+2}']);
                                globals()[f'fig{nh+2}'].savefig(ca_fig_add[nh]);

#             print(f"{n_ep},{num_head} done")
        print(f"Test error under 100 in n_ep = {n_ep}, n_head = {num_head}: {case_test_under_100_n_ep_num_head}/{case_ep_head}")  
print(f"Test set error under 100 in all cases: {case_test_under_100}/{case_num}")

RNN_CA_1DCNN
epoch = 1000, patience = 10
Test error under 100 in n_ep = 1000, n_head = 1: 46/324
Test error under 100 in n_ep = 1000, n_head = 2: 42/324
Test error under 100 in n_ep = 1000, n_head = 3: 44/324
Test error under 100 in n_ep = 1000, n_head = 4: 43/324
Test error under 100 in n_ep = 1000, n_head = 5: 47/324
epoch = 2000, patience = 20
Test error under 100 in n_ep = 2000, n_head = 1: 56/324
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_2000_20\RNN_CA_1DCNN\GRU_1D CNN\ca_batch1\2 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_2000_20\RNN_CA_1DCNN\GRU_1D CNN\ca_batch2\2 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_2000_20\RNN_CA_1DCNN\GRU_1D CNN\ca_batch3\2 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_2000_20\RNN_CA_1DCNN\RNN_1D CNN\ca_batch1\2 h

C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\GRU_1D CNN\ca_batch1\4 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\GRU_1D CNN\ca_batch2\4 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\GRU_1D CNN\ca_batch3\4 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\RNN_1D CNN\ca_batch1\4 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\RNN_1D CNN\ca_batch2\4 heads created
C:\Users\User\Desktop\LIB\CNN work\Variable time length\All dataset\100 cycles\Depth Test_col_3000_500\RNN_CA_1DCNN\RNN_1D CNN\ca_batch3\4 heads created
Test error under 100 in n_ep = 3000, n_head = 4: 154/324
Test error under 100 in n

In [ ]:
history_add, state_dict_add, ta_add, ca_add = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                    rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)

ta_batch1_add, ta_batch2_add, ta_batch3_add, ta_fig_add = ta_per_batches_add(num_cycles, modelname, n_ep, patience,
                                        rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)

ca_batch1_add, ca_batch2_add, ca_batch3_add, ca_fig_add = ca_per_batches_add(num_cycles, modelname, n_ep, patience,
                                        rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)

all_hid1 = hid1 * num_head

model = globals()[modelname](num_vars, num_time, num_cycles, rnn1, bi1, all_hid1, num_head, fil2, pool2, npool2, fsize2, psize2, mids).to(device);
_ = model.load_state_dict(load_data(state_dict_add));

history = load_data(history_add);
rmse = [round(history.iloc[-1, 0],2), round(history.iloc[-1, 1], 2), round(history.iloc[-1, 2], 2)];
case_ep += 1;
if round(history.iloc[-1, 2], 2)<100:
#                     print(f'{n_ep}, {patience}, {rnn1}, {hid1}, {pool2}, {npool2}, {fil2}, {lr}, {rmse}')

    with torch.no_grad():
        i = 0
        fig, axs = plt.subplots(1, 3, figsize = (30, 8));
        for nh in range(num_head):
            globals()[f'fig{nh+2}'], globals()[f'ax{nh+2}'] = plt.subplots(1, 3, figsize = (30, 8));
        rows = 1
        cols = 3
        # ta: (batch_size, num_cycle, timestep)
        b1_ta = torch.empty(batch_size, num_cycles, num_time).to(device)
        b2_ta = torch.empty(batch_size, num_cycles, num_time).to(device)
        b3_ta = torch.empty(batch_size, num_cycles, num_time).to(device)
        tas = [b1_ta, b2_ta, b3_ta]

        # ca shape: [batch_size, num_heads, num_cycle, num_cycle]
        b1_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
        b2_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
        b3_ca = torch.empty(batch_size, num_head, num_cycles, num_cycles).to(device)
        cas = [b1_ca, b2_ca, b3_ca]

        for dataloader in dataloaders:
            for n, (inputs, targets) in enumerate(dataloader):
                inputs = inputs.float().to(device)
                targets = targets.float().to(device)
                # ta: (batch_size, num_cycle, timestep)
                # ca: (batch_size, num_heads, num_cycle, num_cycle)
                yhat, ta, ca = model(inputs)

                if len(ta.size()) !=3:
                    ta = torch.unsqueeze(ta, 0)
                tas[i] = torch.cat((tas[i], ta), dim = 0)

                if len(ca.size()) !=4:
                    ca = torch.unsqueeze(ca, 0)
                cas[i] = torch.cat((cas[i], ca), dim = 0)

            tas[i] = torch.mean(tas[i][batch_size:], dim = 0).squeeze();
            tas[i] = tas[i].detach().cpu().numpy();

            cas[i] = torch.mean(cas[i][batch_size:], dim = 0).squeeze();
            cas[i] = cas[i].detach().cpu().numpy();
        #                                 print(cas[i].type())

            globals()[f'im_{i}'] = axs[i].imshow(tas[i], vmin = 0, vmax = 0.06, cmap = cm.gray);
            _ = axs[i].set_xlabel('Timestep', fontsize = 20);
            for t in axs[i].get_xticklabels():
                t.set_fontsize(20);
            _ = axs[i].set_ylabel('Cycle', fontsize = 20);
            for t in axs[i].get_yticklabels():
                t.set_fontsize(20);

            _ = axs[i].set_title(f'Batch {i+1} temporal attention score', fontsize = 25);

            for nh in range(num_head):
        #                                     print(cas[i][nh,:,:])
                globals()[f'im{nh+2}_{i}'] = globals()[f'ax{nh+2}'][i].imshow(cas[i][nh,:,:], vmin = 0, vmax = 0.06, cmap = cm.gray);
                _ = globals()[f'ax{nh+2}'][i].set_xlabel('Cycle', fontsize = 20);
                for t in globals()[f'ax{nh+2}'][i].get_xticklabels():
                    t.set_fontsize(20);
                _ = globals()[f'ax{nh+2}'][i].set_ylabel('Cycle', fontsize = 20);
                for t in globals()[f'ax{nh+2}'][i].get_yticklabels():
                    t.set_fontsize(20);

                _ = globals()[f'ax{nh+2}'][i].set_title(f'Batch {i+1} cyclic attention score', fontsize = 25);

            i+=1

        fig.subplots_adjust(right = 0.85);
        cbar_ax = fig.add_axes([0.88, 0.15, 0.02, 0.7]);
        for nh in range(num_head):
            globals()[f'fig{nh+2}'].subplots_adjust(right = 0.85);
            globals()[f'cbar_ax{nh+2}'] = globals()[f'fig{nh+2}'].add_axes([0.88, 0.15, 0.02, 0.7]);
        i -= 1
        #     globals()[f'divider_{i}'] = make_axes_locatable(axs[i]);
        #     globals()[f'cax_{i}'] = globals()[f'divider_{i}'].append_axes("right", size="10%", pad=0.05);


        #     globals()[f'cbar_{i}'] = plt.colorbar(globals()[f'im_{i}'], cax = globals()[f'cax_{i}']);
        globals()[f'cbar_{i}'] = plt.colorbar(globals()[f'im_{i}'], cax = cbar_ax);
        for nh in range(num_head):
            globals()[f'cbar{nh+2}_{i}'] = plt.colorbar(globals()[f'im{nh+2}_{i}'], cax = globals()[f'cbar_ax{nh+2}']);
            for t in globals()[f'cbar{nh+2}_{i}'].ax.get_yticklabels():
                t.set_fontsize(20);

        for t in globals()[f'cbar_{i}'].ax.get_yticklabels():
            t.set_fontsize(20);

        _ = fig.suptitle(f'Epoch: {n_ep}, Patience: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Npool: {npool2}, Filter: {fil2}, lr: {lr}, RMSE:{rmse}, n_head = {num_head}', fontsize = 25);
        plt.close(fig)
        fig.savefig(ta_fig_add);

        for nh in range(num_head):
            _ = globals()[f'fig{nh+2}'].suptitle(f'Epoch: {n_ep}, Patience: {patience}, RNN: {rnn1}, Hidden: {hid1}, Pool: {pool2}, Npool: {npool2}, Filter: {fil2}, lr: {lr}, RMSE:{rmse}, n_head = {num_head}, {nh+1}th head', fontsize = 25);
            plt.close(globals()[f'fig{nh+2}'])
            globals()[f'fig{nh+2}'].savefig(ca_fig_add[nh]);

In [41]:
# Hyperparameters
num_vars = 5

modelname = 'RNN_CA_1DCNN'
rnns = ['BiLSTM', 'BiRNN', 'BiGRU']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]
num_heads = [1, 5]

for [n_ep, patience] in ep_pats:
    for num_head in num_heads:
        for rnn1 in rnns:
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
            for hid1 in hids:
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                    for fil2, lr in itertools.product(num_fils, lrs):

                        history_add, _, _ = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)
                        history = load_data(history_add)
    #                     print(history.iloc[-1, :])

                        df.loc[f'{num_head}_{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

            df
            df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{num_head} heads'], 
                            filename = f'{num_head} heads_{modelname}_{rnn1}.csv')
            df.to_csv(df_add)

Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
1_BiLSTM_3_3_1_1_3_0.0001,580.008789,544.864136,524.729431
1_BiLSTM_3_3_1_1_3_0.001,222.946091,134.958069,135.100586
1_BiLSTM_3_3_1_1_3_0.01,202.864075,132.722717,126.011772
1_BiLSTM_3_5_1_1_3_0.0001,561.038452,519.237793,509.160706
1_BiLSTM_3_5_1_1_3_0.001,280.390503,180.089691,180.392090
...,...,...,...
1_BiLSTM_7_5_2_2_3_0.001,311.140808,195.500488,132.094467
1_BiLSTM_7_5_2_2_3_0.01,175.538040,136.696976,99.530930
1_BiLSTM_7_7_2_2_3_0.0001,277.987915,204.732681,193.905685
1_BiLSTM_7_7_2_2_3_0.001,258.023499,191.723633,179.140533


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
1_BiRNN_3_3_1_1_3_0.0001,572.834229,544.519287,529.063049
1_BiRNN_3_3_1_1_3_0.001,270.022369,180.178406,156.324219
1_BiRNN_3_3_1_1_3_0.01,236.343628,155.480606,112.180672
1_BiRNN_3_5_1_1_3_0.0001,570.361145,541.676025,526.225098
1_BiRNN_3_5_1_1_3_0.001,271.651550,161.202911,184.844818
...,...,...,...
1_BiRNN_7_5_2_2_3_0.001,257.717987,166.643448,149.408463
1_BiRNN_7_5_2_2_3_0.01,199.193604,138.462112,95.965057
1_BiRNN_7_7_2_2_3_0.0001,561.898071,519.487854,508.635925
1_BiRNN_7_7_2_2_3_0.001,267.736877,158.551102,125.369530


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


...

,train rmse,val rmse,test rmse
5_BiLSTM_3_3_1_1_3_0.0001,179.936279,100.913155,119.419678
5_BiLSTM_3_3_1_1_3_0.001,125.417572,89.849319,90.010750
5_BiLSTM_3_3_1_1_3_0.01,56.609917,102.718460,69.581139
5_BiLSTM_3_5_1_1_3_0.0001,184.636810,98.483971,146.946136
5_BiLSTM_3_5_1_1_3_0.001,105.070854,80.905945,95.868027
...,...,...,...
5_BiLSTM_7_5_2_2_3_0.001,30.328350,82.083427,82.429268
5_BiLSTM_7_5_2_2_3_0.01,36.895737,97.640396,85.638008
5_BiLSTM_7_7_2_2_3_0.0001,211.418869,101.992813,129.527069
5_BiLSTM_7_7_2_2_3_0.001,89.147842,115.859207,88.014168


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
5_BiRNN_3_3_1_1_3_0.0001,145.500916,91.819588,117.698929
5_BiRNN_3_3_1_1_3_0.001,214.149002,113.729897,130.078506
5_BiRNN_3_3_1_1_3_0.01,137.733307,105.160400,108.616417
5_BiRNN_3_5_1_1_3_0.0001,119.965736,99.917908,100.675652
5_BiRNN_3_5_1_1_3_0.001,87.466629,79.009552,105.270416
...,...,...,...
5_BiRNN_7_5_2_2_3_0.001,30.870829,82.941071,133.322952
5_BiRNN_7_5_2_2_3_0.01,112.826492,102.291267,146.652481
5_BiRNN_7_7_2_2_3_0.0001,116.318588,103.808136,87.323616
5_BiRNN_7_7_2_2_3_0.001,80.782455,86.088966,94.636948


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
5_BiGRU_3_3_1_1_3_0.0001,196.050629,143.983261,130.083221
5_BiGRU_3_3_1_1_3_0.001,86.278259,100.097900,96.513893
5_BiGRU_3_3_1_1_3_0.01,110.732498,108.006500,82.169601
5_BiGRU_3_5_1_1_3_0.0001,574.247437,527.005249,522.174927
5_BiGRU_3_5_1_1_3_0.001,534.174500,477.253113,470.149475
...,...,...,...
5_BiGRU_7_5_2_2_3_0.001,95.598503,119.440872,99.413986
5_BiGRU_7_5_2_2_3_0.01,117.728233,109.409065,67.308899
5_BiGRU_7_7_2_2_3_0.0001,226.367264,129.373962,182.909439
5_BiGRU_7_7_2_2_3_0.001,534.405579,477.517700,470.412354


In [42]:
# Hyperparameters
num_vars = 5

modelname = 'RNN_CA_1DCNN'
rnns = ['LSTM', 'RNN', 'GRU']
hids = [3, 5, 7]
num_time = max_time
num_cycles = 100
ep_pats = [[1000, 10], [2000, 20], [3000, 500]]
lrs = [1e-4, 1e-3, 1e-2]

num_fils = [3, 5, 7]
pools = [1, 2]
npools = [1, 2]
fsize2 = 3
psize2 = 2
mids = [8, 4, 1]
num_heads = [1, 5]

for [n_ep, patience] in ep_pats:
    for num_head in num_heads:
        for rnn1 in rnns:
            bi1 = False
            if rnn1.startswith('Bi'):
                bi1 = True
            df = pd.DataFrame(columns = ['train rmse', 'val rmse', 'test rmse'])
            for hid1 in hids:
                for pool2, npool2 in itertools.product(pools, npools):
                    print(f'Pooling: {pool2} layers, Nonpooling: {npool2} layers')
                    for fil2, lr in itertools.product(num_fils, lrs):

                        history_add, _, _ = history_state_dict_add(num_cycles, modelname, n_ep, patience,
                                                                rnn1, hid1, num_head, fil2, pool2, npool2, fsize2, psize2, lr)
                        history = load_data(history_add)
    #                     print(history.iloc[-1, :])

                        df.loc[f'{num_head}_{rnn1}_{hid1}_{fil2}_{pool2}_{npool2}_{fsize2}_{lr}'] = [history.iloc[-1, 0], history.iloc[-1, 1], history.iloc[-1, 2]]

            df
            df_add = find_directory(back_num = 0, foldername = [f'{num_cycles} cycles', f'Depth Test_col_{n_ep}_{patience}', modelname, f'{num_head} heads'], 
                            filename = f'{num_head} heads_{modelname}_{rnn1}.csv')
            df.to_csv(df_add)

Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
1_LSTM_3_3_1_1_3_0.0001,291.353638,246.916000,266.574493
1_LSTM_3_3_1_1_3_0.001,320.358124,233.256592,207.114914
1_LSTM_3_3_1_1_3_0.01,145.376022,113.580429,96.287247
1_LSTM_3_5_1_1_3_0.0001,552.442627,510.286469,501.341217
1_LSTM_3_5_1_1_3_0.001,229.333313,150.159409,147.967072
...,...,...,...
1_LSTM_7_5_2_2_3_0.001,295.189545,182.175400,156.421234
1_LSTM_7_5_2_2_3_0.01,254.809189,144.666214,140.973053
1_LSTM_7_7_2_2_3_0.0001,313.966949,250.656769,238.438065
1_LSTM_7_7_2_2_3_0.001,249.343628,148.890533,132.367004


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
1_RNN_3_3_1_1_3_0.0001,572.149902,543.800598,528.325928
1_RNN_3_3_1_1_3_0.001,300.422272,192.311035,171.067047
1_RNN_3_3_1_1_3_0.01,271.162415,151.072525,160.051788
1_RNN_3_5_1_1_3_0.0001,572.669189,544.360046,528.868469
1_RNN_3_5_1_1_3_0.001,273.154633,171.190048,117.860489
...,...,...,...
1_RNN_7_5_2_2_3_0.001,278.259918,164.139786,146.933197
1_RNN_7_5_2_2_3_0.01,202.130676,126.842285,106.032379
1_RNN_7_7_2_2_3_0.0001,308.129333,247.837921,210.682068
1_RNN_7_7_2_2_3_0.001,289.556549,191.325516,144.650742


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


...

,train rmse,val rmse,test rmse
5_LSTM_3_3_1_1_3_0.0001,152.895111,96.314613,142.735565
5_LSTM_3_3_1_1_3_0.001,97.908295,82.197945,112.729599
5_LSTM_3_3_1_1_3_0.01,95.233780,96.535591,102.068825
5_LSTM_3_5_1_1_3_0.0001,234.181396,113.165817,138.902969
5_LSTM_3_5_1_1_3_0.001,114.461380,96.178513,119.048218
...,...,...,...
5_LSTM_7_5_2_2_3_0.001,18.129538,79.930313,79.531639
5_LSTM_7_5_2_2_3_0.01,65.447121,93.163773,90.460258
5_LSTM_7_7_2_2_3_0.0001,124.843643,103.459892,92.653473
5_LSTM_7_7_2_2_3_0.001,40.520958,100.991028,102.420792


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
5_RNN_3_3_1_1_3_0.0001,156.883057,83.099594,124.316795
5_RNN_3_3_1_1_3_0.001,99.324875,61.904739,133.750702
5_RNN_3_3_1_1_3_0.01,67.985550,96.359093,101.835251
5_RNN_3_5_1_1_3_0.0001,168.476486,69.802505,126.858444
5_RNN_3_5_1_1_3_0.001,123.325836,85.294266,111.394997
...,...,...,...
5_RNN_7_5_2_2_3_0.001,33.991856,86.124771,98.230621
5_RNN_7_5_2_2_3_0.01,105.659851,98.885864,117.622681
5_RNN_7_7_2_2_3_0.0001,237.636475,118.808784,131.347809
5_RNN_7_7_2_2_3_0.001,54.092216,73.802643,103.960686


Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers
Pooling: 1 layers, Nonpooling: 1 layers
Pooling: 1 layers, Nonpooling: 2 layers
Pooling: 2 layers, Nonpooling: 1 layers
Pooling: 2 layers, Nonpooling: 2 layers


,train rmse,val rmse,test rmse
5_GRU_3_3_1_1_3_0.0001,220.819244,115.364960,134.843811
5_GRU_3_3_1_1_3_0.001,71.524925,90.553947,81.513290
5_GRU_3_3_1_1_3_0.01,94.345932,75.077995,76.015289
5_GRU_3_5_1_1_3_0.0001,156.401199,77.167572,112.889954
5_GRU_3_5_1_1_3_0.001,94.096352,104.860062,77.405327
...,...,...,...
5_GRU_7_5_2_2_3_0.001,98.737206,126.263290,80.277527
5_GRU_7_5_2_2_3_0.01,44.408699,81.474823,82.105980
5_GRU_7_7_2_2_3_0.0001,168.745605,107.569550,126.595947
5_GRU_7_7_2_2_3_0.001,121.815231,114.240730,78.488541
